# I.  Text Cleaning and Preparation:  Introduction

Some basic code for text cleaning and prep

# II.  Setup the Environment

We'll focus on usingthe NLTK package to clean and process our text for final analysis.  Comments in the code identify each of the packages being loaded.  In each case, you can refer to the package documentation for more specific information about the package being used.  You must run the code cells below to properly prepare your environment to perfrom the text mining and analysis tasks presented in this module.

In [1]:
# update collab environment to latest version of NLTK
# documentation: https://www.nltk.org/
!pip install nltk -U

     |████████████████████████████████| 1.5MB 6.5MB/s 
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [2]:
# update to the latest version fo Spacy
# https://spacy.io/
!pip install spacy -U

     |████████████████████████████████| 6.4MB 5.0MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 624kB 41.3MB/s 
     |████████████████████████████████| 10.1MB 30.5MB/s 
     |████████████████████████████████| 460kB 32.6MB/s 
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [3]:
# import the base nltk package
import nltk

# load the nltk tokenize module
from nltk.tokenize import word_tokenize

# download the punkt model
nltk.download('punkt')

# import nltk stopword module
from nltk.corpus import stopwords

# donload the stopword list
nltk.download('stopwords')

# import the nltk porter stemmer
from nltk.stem.porter import PorterStemmer

# import the nltk lemmatizer
from nltk.stem import WordNetLemmatizer

# download nltk wordnet model
nltk.download('wordnet')

# import the regular expressions package
import re

# inport the string package
import string

# import Spacy
import spacy

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [4]:
# download and install the spacy language model
!python -m spacy download en_core_web_sm
sp=spacy.load('en_core_web_sm')

2021-07-14 17:47:01.023157: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 13.6MB 215kB/s 
  Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


# III.  Load the Text File

In [5]:
from google.colab import drive
drive.mount('/gdrive/')

Mounted at /gdrive/


In [6]:
working_file_path = "/gdrive/MyDrive/rbs_digital_approaches_2021/data_class/melville.txt"

Now that you've defined a file to load, we can open the file and read its contents into a string variable.

In [7]:
# open a text file for processing
working_file = open(working_file_path, "r")

# read the file contents into a string variable
working_text = working_file.read()

You can check that your file loaded by checking the length and examining the opening characters of the working_text variable.

In [15]:
# print the character length of our working text
# and the first several characters
print('Characters in string:', len(working_text))
print(working_text[:600:1])

Characters in string: 1230802
The Project Gutenberg EBook of Moby Dick; or The Whale, by Herman Melville This eBook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.org Title: Moby Dick; or The Whale Author: Herman Melville Last Updated: January 3, 2009 Posting Date: December 25, 2008 [EBook #2701] Release Date: June, 2001 Language: English *** START OF THIS PROJECT GUTENBERG EBOOK MOBY DICK; OR THE WHALE *** Produced by Daniel Lazarus and Jo


IV.  Convert to Lowercase

# IV.  Remove Newline Characters and Strip Spaces

In [11]:
# define a pattern for finding newlines
pattern = re.compile(r"\n", re.DOTALL | re.MULTILINE | re.IGNORECASE)
# run the replacement.  
working_text = re.sub(pattern, " ", working_text)

In [13]:
# define a patern for finding multiple spaces
pattern = re.compile(r"\s+")
# run the replacement
working_text = re.sub(pattern, " ", working_text)

In [14]:
# strip leading and trailing spaces
working_text = working_text.strip()

# IV. Remove Editorial Text

In [16]:
# look at the first 600 characters of the string
print(len(working_text))
print(working_text[:600:1])

1230802
The Project Gutenberg EBook of Moby Dick; or The Whale, by Herman Melville This eBook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.org Title: Moby Dick; or The Whale Author: Herman Melville Last Updated: January 3, 2009 Posting Date: December 25, 2008 [EBook #2701] Release Date: June, 2001 Language: English *** START OF THIS PROJECT GUTENBERG EBOOK MOBY DICK; OR THE WHALE *** Produced by Daniel Lazarus and Jo


In [18]:
# define a pattern and remove the opening text
pattern = re.compile(r"^.*chapter 1\. Loomings\.?", re.IGNORECASE)
working_text = re.sub(pattern, "", working_text)

In [19]:
# define a pattern and remove the closing text
pattern = re.compile(r"End of Project Gutenberg's.*", re.IGNORECASE)
working_text = re.sub(pattern, "", working_text)

In [20]:
# look at the first 600 characters of the string
print(len(working_text))
print(working_text[1190000:1190046:1])

1190046
 missing children, only found another orphan. 


# V. Save Clean Blob Version

It's a good idea to put aside a version of the minimally claeaned text as single blob for use later. 

In [21]:
blob_text = working_text

# For the rest of the cleaning we'll tokenize and then clean because NLTK Likes it that way

In [22]:
# tokenize on words
tokens = word_tokenize(working_text)


In [ ]:
# look at the results
print(tokens[:10])

In [23]:
# remove punctuation from each word
table = str.maketrans('', '', string.punctuation)
filtered_tokens = [w.translate(table) for w in tokens]

In [24]:
# look at the results
print(filtered_tokens[:10])

['Call', 'me', 'Ishmael', '', 'Some', 'years', 'ago', '', 'never', 'mind']


In [25]:
# remove remaining tokens that are not alphabetic
filtered_tokens = [word for word in filtered_tokens if word.isalpha()]

In [26]:
# look at the results
print(filtered_tokens[:10])

['Call', 'me', 'Ishmael', 'Some', 'years', 'ago', 'never', 'mind', 'how', 'long']


In [27]:
# load the nltp stopword list
stop_words = set(stopwords.words('english'))


In [28]:
# review the stopwords
print(stop_words)

{'our', 'between', 'be', 'against', 'didn', 'm', 'hers', 'ourselves', 'from', "hadn't", 'nor', 'having', 'won', 'myself', 'which', 'few', "you've", 'at', 'that', 'ours', 'what', 'me', 'when', 'an', 'should', 've', 'up', 'am', 'some', 'here', 'my', 'only', 's', 'in', 'or', 'not', 'for', 'his', 'we', 'doesn', "haven't", 'so', 'than', 'wouldn', 'each', 'o', 'with', 'then', 'he', "doesn't", 'himself', 'over', "weren't", 'been', 'through', 'you', "she's", 'no', "didn't", 'needn', 'where', 'had', "mightn't", 'all', "aren't", 'during', 'other', 'once', 'couldn', 'will', 'theirs', 'its', 'very', "you'd", 'whom', 'she', 'shan', 'these', 'mustn', 'of', 'why', 'until', 'again', "should've", 'if', 'just', "that'll", 'about', 'by', 'are', 'while', 'i', 'out', 'before', 'your', 'isn', "couldn't", 'does', 'them', 'below', 'y', 'itself', 'ma', "mustn't", 'shouldn', "wasn't", "you'll", 'herself', 'who', 'after', 'same', 'this', 'how', 'll', 'down', 'own', 'can', 'such', 'above', 'is', 'and', 'did', 'an

Note that based on your research qustion you might want to modify the stopword list.  You can reuse code from above (for removing spaces, etc.) to create a list of words you want to remoove from the stopword list.  Alternatively, you can append other words to this list or build your own from scratch.

In [29]:
# remove the stopwords
filtered_tokens = [w for w in filtered_tokens if not w in stop_words]

In [30]:
# look at the results
print(filtered_tokens[:10])

['Call', 'Ishmael', 'Some', 'years', 'ago', 'never', 'mind', 'long', 'precisely', 'little']


# Stemming



Stemmking uses a rules-based algorithm to remove plural endings, "ing" endings and the like from words as a means of reducing variation.  [See the Wikipedia article here for a good overview](https://en.wikipedia.org/wiki/Stemming).

In [31]:
# instantiate a porter stemmer class object
p_stemmer = PorterStemmer()

#  run the stemmer on our list of filtered words
stemmed_tokens = [p_stemmer.stem(word) for word in filtered_tokens]

In [32]:
# view the results
print(stemmed_tokens[:100])

['call', 'ishmael', 'some', 'year', 'ago', 'never', 'mind', 'long', 'precis', 'littl', 'money', 'purs', 'noth', 'particular', 'interest', 'shore', 'i', 'thought', 'i', 'would', 'sail', 'littl', 'see', 'wateri', 'part', 'world', 'it', 'way', 'i', 'drive', 'spleen', 'regul', 'circul', 'whenev', 'i', 'find', 'grow', 'grim', 'mouth', 'whenev', 'damp', 'drizzli', 'novemb', 'soul', 'whenev', 'i', 'find', 'involuntarili', 'paus', 'coffin', 'warehous', 'bring', 'rear', 'everi', 'funer', 'i', 'meet', 'especi', 'whenev', 'hypo', 'get', 'upper', 'hand', 'requir', 'strong', 'moral', 'principl', 'prevent', 'deliber', 'step', 'street', 'method', 'knock', 'peopl', 'hat', 'i', 'account', 'high', 'time', 'get', 'sea', 'soon', 'i', 'thi', 'substitut', 'pistol', 'ball', 'with', 'philosoph', 'flourish', 'cato', 'throw', 'upon', 'sword', 'i', 'quietli', 'take', 'ship', 'there', 'noth']


# Lemmatizing

Lemmatization is an NLP based reduction method that uses language models to reduce all variants of word (is, are, am, etc.) to a common linguistic root (be).  It generally improves the quality of semantic models because it reduces lexical variation in favor of semantic sameness.  However, in many cases we care a lot about particular words, and stemming often erases these differences, so think carefully before stemming.

In [33]:
sp_text = sp(blob_text[:100:1])
for word in sp_text:
  print(word.text, word.lemma, word.lemma_)


  8532415787641010193  
Call 14229572451745258962 call
me 4690420944186131903 I
Ishmael 9700595093220607158 Ishmael
. 12646065887601541794 .
Some 7000492816108906599 some
years 14889849580704678361 year
ago 18432988265768819336 ago
-- 10501404726543969396 --
never 10879546028211794123 never
mind 8811887512194128535 mind
how 16331095434822636218 how
long 12965068231793614765 long
precisely 17167850068121054515 precisely
-- 10501404726543969396 --
having 14692702688101715474 have
little 9778055143417507723 little
or 3740602843040177340 or
no 13055779130471031426 no
money 14917444839815175757 money
in 3002984154512732771 in
my 227504873216781231 my
pur 9444015941227026890 pur
